In [1]:
from micromind import MicroMind, Metric
from micromind.networks import PhiNet
from micromind.utils.parse import parse_arguments

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from dataset import ImageWoof
from matplotlib import pyplot as plt

batch_size = 128

class ImageClassification(MicroMind):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.modules["classifier"] = PhiNet(
            (3, 32, 32), include_top=True, num_classes=10, alpha=2
        )

    def forward(self, batch):
        return self.modules["classifier"](batch[0])

    def compute_loss(self, pred, batch):
        return nn.CrossEntropyLoss()(pred, batch[1])


if __name__ == "__main__":
    hparams = parse_arguments()
    m = ImageClassification(hparams)

    def compute_accuracy(pred, batch):
        tmp = (pred.argmax(1) == batch[1]).float()
        return tmp

    transform = transforms.Compose(
        [transforms.ToTensor(), 
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
         transforms.Resize((160, 160), antialias=True)
        ]
    )

    trainset = ImageWoof(
        root=".", train=True, transform=transform, img_size=160
    )
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=batch_size, shuffle=True, num_workers=1
    )

    testset = ImageWoof(
        root=".", train=False, transform=transform, img_size=160
    )
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=batch_size, shuffle=False, num_workers=1
    )

    """
    # show some images
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    plt.figure(figsize=(16, 6))
    for i in range(10):
        plt.subplot(2, 5, i + 1)
        image, _ = trainloader.dataset.__getitem__(i)
        image= image.permute(1,2,0).numpy()
        image = (image*std)+mean
        plt.imshow(image)

    plt.show()    
    plt.waitforbuttonpress()
    """

    acc = Metric(name="accuracy", fn=compute_accuracy)

    m.train(
        epochs=300,
        datasets={"train": trainloader, "val": testloader, "test": testloader},
        metrics=[acc],
        debug=hparams.debug,
    )

    m.test(
        datasets={"test": testloader},
    )

    m.export("output_onnx", "onnx", (3, 32, 32))


usage: ipykernel_launcher.py [-h] [--lr LR] [--optimizer {adam,sgd}]
                             [--experiment_name EXPERIMENT_NAME]
                             [--output_folder OUTPUT_FOLDER] [--debug]
ipykernel_launcher.py: error: unrecognized arguments: --ip=127.0.0.1 --stdin=9003 --control=9001 --hb=9000 --Session.signature_scheme="hmac-sha256" --Session.key=b"1baef392-3fc4-4392-9b43-b045a51de874" --shell=9002 --transport="tcp" --iopub=9004 --f=/Users/sebastiancavada/Library/Jupyter/runtime/kernel-v2-4084AxnTDBkFnskQ.json


SystemExit: 2

/Users/sebastiancavada/anaconda3/envs/ai701/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
